In [20]:
from dotenv import load_dotenv
import os
from pinecone import Pinecone, ServerlessSpec

# Load environment variables from a .env file
load_dotenv()





True

In [21]:
pc = Pinecone(api_key= os.getenv("PINECONE_API_KEY"))
try:
    pc.create_index(
        name="rag",
        dimension = 1024,
        metric="cosine",
        spec=ServerlessSpec(cloud="aws",region="us-east-1")
    )
except:
    pass 

In [22]:
import json
data = json.load(open("reviews.json"))

data['reviews']

[{'name': 'Alice Johnson',
  'skills': 'Python, Data Analysis',
  'stars': 5,
  'comments': 'This project was exceptionally well done. The code is clean, and the analysis is insightful.'},
 {'name': 'Bob Smith',
  'skills': 'JavaScript, React',
  'stars': 4,
  'comments': 'Solid project with great attention to detail, but the UI could use some enhancements.'},
 {'name': 'Carol Williams',
  'skills': 'Java, Spring Boot',
  'stars': 4,
  'comments': 'The backend logic is strong, but there are some issues with error handling.'},
 {'name': 'David Brown',
  'skills': 'C++, Embedded Systems',
  'stars': 5,
  'comments': 'Impressive work, especially in optimizing for performance. Very well done!'},
 {'name': 'Eva Green',
  'skills': 'HTML, CSS, UX Design',
  'stars': 3,
  'comments': 'The design is visually appealing, but the responsiveness needs improvement.'},
 {'name': 'Frank Harris',
  'skills': 'Ruby on Rails, SQL',
  'stars': 5,
  'comments': 'Excellent use of Rails for a clean and effi

In [23]:
from mixedbread_ai.client import MixedbreadAI
processed_data=[]

from openai import OpenAI

# Initialize OpenAI client using OpenRouter's base URL and API key from environment variable
mxbai = MixedbreadAI(api_key = os.getenv('MIXEDBREAD_API_KEY'))

for review in data['reviews'] :
    response = mxbai.embeddings(
        input = review['comments'],
        model = "mixedbread-ai/mxbai-embed-large-v1",
        normalized = True,
        encoding_format = 'float',
        truncation_strategy = 'end'
    )
    embedding = response.data[0].embedding
    processed_data.append({
        'values': embedding,
        'id' : review['name'],
        'metadata': {
            'review' : review['comments'],
            'skills': review['skills'],
            'stars': review['stars'] 
        }   
    })

In [24]:
processed_data[0]

{'values': [0.0012989044,
  0.042877197,
  -0.009086609,
  -0.0014533997,
  -0.022384644,
  -0.012001038,
  -0.0049972534,
  0.010795593,
  0.033416748,
  0.019638062,
  0.0037250519,
  -0.0076179504,
  0.0066337585,
  -0.05255127,
  -0.03878784,
  -0.04663086,
  -0.018051147,
  -0.0054626465,
  -0.0715332,
  0.024459839,
  0.002254486,
  0.013549805,
  -0.06518555,
  -0.057891846,
  -0.039642334,
  0.05621338,
  0.025848389,
  -0.0058555603,
  0.07684326,
  0.059051514,
  0.008544922,
  0.034118652,
  -0.0053710938,
  -0.04812622,
  0.013259888,
  -0.010414124,
  0.060302734,
  -0.0074882507,
  0.009735107,
  -0.048461914,
  0.011245728,
  -0.013175964,
  0.018478394,
  -0.081604004,
  -0.07824707,
  -0.012268066,
  0.017608643,
  -0.0042381287,
  0.02255249,
  -0.034576416,
  -0.0017690659,
  0.032440186,
  0.039276123,
  -0.044006348,
  -0.002784729,
  0.0020256042,
  0.0044441223,
  0.01285553,
  -0.045837402,
  0.035491943,
  0.033477783,
  -0.00983429,
  -0.0018157959,
  -0.06829

In [25]:
index = pc.Index('rag')
index.upsert(
    vectors= processed_data,
    namespace = 'ns1'
)

{'upserted_count': 21}